In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set(font_scale = 2)
sns.set_style("whitegrid")

In [ ]:
markers = ["s","*","o","^","v",">","D"]

In [ ]:
def handle_img(fig_id, outside=False, img_dir="/mnt/idms/fberes/data/bitcoin_ln_research/results/", anchor=(1.02, 1.02)):
    if outside:
        lgd = plt.legend(loc=2,bbox_to_anchor=anchor)
        plt.savefig("%s/%s.pdf" % (img_dir, fig_id), format="pdf", bbox_extra_artists=(lgd,), bbox_inches='tight')
    else:
        lgd = plt.legend(loc=8, ncol=2)
        plt.savefig("%s/%s.pdf" % (img_dir, fig_id), format="pdf")

def plot_col_wrt_tx_amount(df, value_col, nodes, key_col="amount", fig_size=(15,10)):
    plt.figure(figsize=fig_size)
    for i, n in enumerate(nodes):
        node_df = df[df["name"]==n]
        x, y = node_df[key_col], node_df[value_col]
        ls = "-" if i < len(markers) else "--" 
        plt.plot(x, y, marker=markers[i%len(markers)], markersize=15, linewidth=3, linestyle=ls, label=n)
    if key_col == "amount":
        plt.xlabel(r"$\alpha$ (satoshi)")
        plt.xscale("log")
    else:
        plt.xlabel("epsilon")
    if value_col == "income":
        plt.ylabel("income (satoshi)")
    else:
        plt.ylabel(value_col)
    plt.yscale("log")
    
def calculate_roi_for_routers(node_efficiency, router_1ML_data):
    router_data = router_1ML_data.copy().merge(node_efficiency, on="name")
    router_data["annual_roi"] = router_data.apply(lambda x: x["daily_income"]*365.0/x["cap_sat"], axis=1)
    router_data["dupl_factor_5%_roi"] = router_data.apply(lambda x: (x["cap_sat"]*0.05)/(x["daily_income"]*365.0), axis=1)
    router_data["total_fee_5%_roi"] = router_data["total_fee"] * router_data["dupl_factor_5%_roi"]
    router_data["annual_roi_rank"] = router_data["annual_roi"].rank(ascending=False).astype("int64")
    router_data["total_fee_rank"] = router_data["total_fee"].rank(ascending=False).astype("int64")
    router_data["daily_traffic_rank"] = router_data["daily_traffic"].rank(ascending=False).astype("int64")
    router_data = router_data.sort_values("daily_income",ascending=False).reset_index(drop=True)
    return router_data

In [ ]:
node_name_replacements = {
    "rompert.com≡ƒö╡":"rompert.com",
    "03021c5f5f57322740e4":"BlueWallet",
    "≡ƒÜÇ≡ƒîæ BOLTENING.club":"BOLTENING.club"
}

# 1. Node traffic and income

In [ ]:
nodes = ["rompert.com","LNBIG.com","zigzag.io","yalls.org","ln1.satoshilabs.com","tippin.me","ACINQ","LightningPowerUsers.com","1ML.com node ALPHA","LightningTo.Me"]

In [ ]:
ipt_col = "income per transaction (satoshi)"

## i.) with respect to amount

drop_disabled = True
sim_dirs = {
10000:"2019-09-07_10:23:25_10000sat_k6000",
25000:"2019-09-07_05:06:14_25000sat_k6000",
50000:"2019-09-06_22:03:19_50000sat_k6000",
100000:"2019-09-07_14:24:11_100000sat_k6000",
200000:"2019-09-07_18:37:32_200000sat_k6000",
500000:"2019-09-07_22:40:54_500000sat_k6000",
}
root_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/experiments_with_rayleigh/results/router_traffic_and_income/1days_40snapshot/"

sim_dirs = {
10000:"10000sat_k7000_eps0.80_selected",
20000:"20000sat_k7000_eps0.80_selected",
60000:"60000sat_k7000_eps0.80_selected",
100000:"100000sat_k7000_eps0.80_selected",
200000:"200000sat_k7000_eps0.80_selected",
500000:"500000sat_k7000_eps0.80_selected",
}
root_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/lnbig_tuning_1days//"

In [ ]:
sim_dirs = {
60000:"60000sat_k7000_eps0.80_wdepFalse_selected"
}
root_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/without_depletion//"

In [ ]:
parts = []
for amount, folder in sim_dirs.items():
    df = pd.read_csv("%s/%s.csv" % (root_dir, folder))
    df["amount"] = amount
    parts.append(df.replace(node_name_replacements).rename({"mean_fee":"income","mean_num_trans":"traffic"},axis=1))
    #parts.append(df.replace(node_name_replacements).rename({"fee":"income","num_trans":"traffic"},axis=1))
traffic_and_income_df = pd.concat(parts)

## Data Description

- income: total routing income for the given entity
- traffic: number of transactions routed through the given entity
- name: name of the entity (for most of them it is a single node BUT for LNBIG.com it is approx. 25 node)
- amount: the amount of each transaction in a given simulation

In [ ]:
traffic_and_income_df.head()

## Results

### a.) Routing income with respect to tx_amount

In [ ]:
plot_col_wrt_tx_amount(traffic_and_income_df, "income", nodes)
handle_img("router_income", outside=True, img_dir=root_dir)

### b.) Routing traffic with respect to tx_amount

In [ ]:
plot_col_wrt_tx_amount(traffic_and_income_df, "traffic", nodes)
handle_img("router_traffic", outside=True, img_dir=root_dir)

### c.) Routing income per transaction (calculated from routed traffic) with respect to tx_amount

In [ ]:
traffic_and_income_df[ipt_col] = traffic_and_income_df["income"] / traffic_and_income_df["traffic"]

In [ ]:
plot_col_wrt_tx_amount(traffic_and_income_df, ipt_col, nodes)
handle_img("node_income_per_transaction", outside=True, img_dir=root_dir)

## ii.) With respect to epsilon

root_dir_eps = "/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/effect_of_epsilon/"
sim_dirs_eps = {
0.0:"2019-09-26_14:15:21_50000sat_k6000_eps0.00_selected",
0.2:"2019-09-26_14:15:21_50000sat_k6000_eps0.20_selected",
0.5:"2019-09-26_14:15:21_50000sat_k6000_eps0.50_selected",
0.8:"2019-09-26_14:15:21_50000sat_k6000_eps0.80_selected",
1.0:"2019-09-26_14:15:21_50000sat_k6000_eps1.00_selected",
}

root_dir_eps = "/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/effect_of_epsilon/"
sim_dirs_eps = {
0.0:"60000sat_k7000_eps0.00_selected",
0.2:"60000sat_k7000_eps0.20_selected",
0.5:"60000sat_k7000_eps0.50_selected",
0.8:"60000sat_k7000_eps0.80_selected",
1.0:"60000sat_k7000_eps1.00_selected",
}

parts = []
for eps, folder in sim_dirs_eps.items():
    df = pd.read_csv("%s/%s.csv" % (root_dir_eps, folder))
    df["epsilon"] = eps
    parts.append(df.replace(node_name_replacements).rename({"mean_fee":"income","mean_num_trans":"traffic"},axis=1))
traffic_and_income_df_eps = pd.concat(parts)

## Results

### a.) Routing income with respect to tx_amount

plot_col_wrt_tx_amount(traffic_and_income_df_eps, "income", nodes, key_col="epsilon")
handle_img("router_income_eps", outside=True, img_dir=root_dir_eps)

### b.) Routing traffic with respect to tx_amount

plot_col_wrt_tx_amount(traffic_and_income_df_eps, "traffic", nodes, key_col="epsilon")
handle_img("router_traffic_eps", outside=True, img_dir=root_dir_eps)

### c.) Routing income per transaction (calculated from routed traffic) with respect to tx_amount

ipt_col = "income per transaction (SAT)"

traffic_and_income_df_eps[ipt_col] = traffic_and_income_df_eps["income"] / traffic_and_income_df_eps["traffic"]

plot_col_wrt_tx_amount(traffic_and_income_df_eps, ipt_col, nodes, key_col="epsilon")
handle_img("node_income_per_transaction_eps", outside=True, img_dir=root_dir_eps)

# 2. ROI analysis

In [ ]:
import sys
sys.path.insert(0,"./python")
from analysis_utils import relevant_routers

In [ ]:
other_routers = relevant_routers()

## Load meta data from 1ML.com

In [ ]:
node_names = pd.read_csv("/mnt/idms/fberes/data/bitcoin_ln_research/node_names.csv")

In [ ]:
lnbig_keys = list((node_names[node_names["is_lnbig"]]["pub_key"]))

## Load pricing information

In [ ]:
selected_keys = other_routers.copy()

In [ ]:
selected_keys += lnbig_keys

In [ ]:
all_edges = pd.read_csv("/mnt/idms/fberes/data/bitcoin_ln_research/directed_graphs/directed_temporal_multi_edges_1days.csv")

In [ ]:
edges = all_edges[all_edges["snapshot_id"]<40]

In [ ]:
edges = edges[edges["trg"].isin(selected_keys)]

In [ ]:
edges["total_fee"] = edges["fee_base_msat"]/1000.0 + 60000*edges["fee_rate_milli_msat"]/10.0**6

In [ ]:
node_routing_costs = edges.groupby("trg")[["fee_base_msat","fee_rate_milli_msat","total_fee"]].mean().reset_index()

## Merge meta and pricing data

In [ ]:
node_merged_data = node_names.merge(node_routing_costs, left_on="pub_key", right_on="trg").drop(["trg"], axis=1)

In [ ]:
node_merged_data.head()

In [ ]:
LNBIG_data = node_merged_data[node_merged_data["is_lnbig"]]
len(LNBIG_data)

In [ ]:
LNBIG_record = {
    "name":"LNBIG.com",
    "cap_ratio":LNBIG_data["cap_ratio"].sum(),
    "cap_sat":LNBIG_data["cap_sat"].sum(),
    "fee_base_msat":LNBIG_data["fee_base_msat"].mean(),
    "fee_rate_milli_msat":LNBIG_data["fee_rate_milli_msat"].mean(),
    "total_fee":LNBIG_data["total_fee"].mean(),
}

LNBIG_record = LNBIG_data[["cap_ratio","cap_sat"]].sum()
LNBIG_record["name"] = "LNBIG.com"

In [ ]:
router_node_data = node_merged_data[node_merged_data["pub_key"].isin(other_routers)][["name","cap_ratio","cap_sat","fee_base_msat","fee_rate_milli_msat","total_fee"]].replace(node_name_replacements)

In [ ]:
router_node_data = router_node_data.append(LNBIG_record, ignore_index=True)

router_node_data.sort_values("cap_ratio", ascending=False).head()

## Calculate ROI based on simulation (old version)

- number of daily transactions: 5000
- tx_amount: 50K SAT (~5,2 USD)

node_efficiency = traffic_and_income_df[traffic_and_income_df["amount"]==50000][["name","income","traffic"]].rename({"income":"daily_income","traffic":"daily_traffic"}, axis=1)

In [ ]:
node_efficiency = traffic_and_income_df[traffic_and_income_df["amount"]==60000][["name","income","traffic"]].rename({"income":"daily_income","traffic":"daily_traffic"}, axis=1)

In [ ]:
router_node_data_with_roi = calculate_roi_for_routers(node_efficiency, router_node_data)

In [ ]:
for col in ["dupl_factor_5%_roi","total_fee","total_fee_5%_roi","daily_income","daily_traffic"]:
    router_node_data_with_roi[col] = np.around(router_node_data_with_roi[col], decimals=1)#.astype("int64")

In [ ]:
router_node_data_with_roi = router_node_data_with_roi.drop(["fee_base_msat","fee_rate_milli_msat"], axis=1)

In [ ]:
router_node_data_with_roi

## Calculate ROI with epsilon

node_efficiency_eps = traffic_and_income_df_eps[traffic_and_income_df_eps["epsilon"]==0.8][["name","income","traffic"]].rename({"income":"daily_income","traffic":"daily_traffic"}, axis=1)

router_node_data_with_roi_eps = calculate_roi_for_routers(node_efficiency_eps, router_node_data)

router_node_data_with_roi_eps

## Export results

In [ ]:
router_node_data_with_roi.to_csv("%s/roi_analysis.csv" % root_dir, index=False)

In [ ]:
s = str(router_node_data_with_roi.to_latex(index=False))

In [ ]:
s = s.replace("name","Entity name")
s = s.replace("cap\_ratio","Cap. ratio")
s = s.replace("cap\_sat","Entity capacity")
s = s.replace("daily\_income","Daily income")
s = s.replace("daily\_traffic","Daily traffic")
s = s.replace("annual\_roi","Annual ROI")
s = s.replace("dupl\_factor\_5\%\_roi","Duplication factor")
s = s.replace("Annual ROI\_rank","ROI rank")
s = s.replace("Daily traffic\_rank","Traffic rank")

In [ ]:
print(s)

# 3. Simulation stability

In [ ]:
epsilons = [0.0,0.2,0.5,0.8,1.0]

In [ ]:
folder_prefix = "/mnt/idms/fberes/data/bitcoin_ln_research/results/simulation_stability/effect_of_epsilon"

In [ ]:
stability_dfs = {}
for eps in epsilons:
    stability_dfs[eps] = pd.read_csv("%s/60000sat_k7000_eps%.2f_stability_res.csv" % (folder_prefix, eps))
    stability_dfs[eps] = stability_dfs[eps][stability_dfs[eps]["correlation type"]!="pearson"]
    stability_dfs[eps] = stability_dfs[eps].replace({"sender transaction fee":"sender fee"})

In [ ]:
output_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/results/simulation_stability"
for eps in epsilons:
    sns.catplot(data=stability_dfs[eps], x="correlation type", y="value", hue="statistics", kind="bar", height=8)
    plt.title(r"$\epsilon$=%.1f" % eps)
    plt.savefig("%s/60000sat_k7000_eps%.2f_cross_correlation.pdf" % (output_dir,eps), format="pdf")

# 4. Income correlation for different epsilons

In [ ]:
income_dir_prefix = "/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income"
all_node_incomes = {}
for eps in epsilons:
    #all_node_incomes[eps] = pd.read_csv(income_dir_prefix+"/effect_of_epsilon/2019-09-26_14:15:21_50000sat_k6000_eps%.2f_all.csv" % eps)
    all_node_incomes[eps] = pd.read_csv(income_dir_prefix+"/effect_of_epsilon/60000sat_k7000_eps%.2f_all.csv" % eps)

In [ ]:
all_node_incomes[0.0].head()

In [ ]:
from ln_utils import corr_mx

cmap = "RdBu_r"
fig_s=(8,8)

def merge_and_plot_corr_mx(all_node_incomes, value_col):
    eps = 0.0
    merged_incomes = all_node_incomes[eps][["node",value_col]].rename({value_col:eps}, axis=1)
    for eps in [0.2,0.5,0.8,1.0]:
        merged_incomes = merged_incomes.merge(all_node_incomes[eps][["node",value_col]].rename({value_col:eps}, axis=1), on="node", how="outer")
    print(merged_incomes.isnull().sum())
    merged_incomes = merged_incomes.fillna(0.0)
    merged_incomes_wout_ids = merged_incomes.drop("node",axis=1)
    for corr_type in ["spearman","kendall","wkendall"]:
        fig, ax = plt.subplots(figsize=fig_s)
        plt.title(corr_type)
        sns.heatmap(corr_mx(merged_incomes_wout_ids, method=corr_type), annot=True, fmt='.2f', vmin=0.65, vmax=1.0, ax=ax, cmap=cmap, square=True)
        ax.set_ylim(6.0, 0)
        plt.savefig("%s/60000sat_k7000_%s_%s.pdf" % (income_dir_prefix, value_col, corr_type), format="pdf")

In [ ]:
merge_and_plot_corr_mx(all_node_incomes, "mean_fee")

In [ ]:
merge_and_plot_corr_mx(all_node_incomes, "mean_num_trans")

# 5. Entity global failure ratios

In [ ]:
global_failure_ratios = pd.read_csv("/mnt/idms/fberes/data/bitcoin_ln_research/results/entity_global_failure_ratios.csv")

In [ ]:
threshold = float(global_failure_ratios[global_failure_ratios["entity"]=="all"]["failure_ratio"])

In [ ]:
threshold

In [ ]:
global_failure_ratios = global_failure_ratios[global_failure_ratios["entity"]!="all"]

In [ ]:
global_failure_ratios = global_failure_ratios.replace(node_name_replacements)

In [ ]:
plt.figure(figsize=(16,8))
plt.bar(global_failure_ratios["entity"], global_failure_ratios["failure_ratio"])
plt.xticks(rotation=90)
plt.ylim((0.35,0.393))
plt.xlim((-1.0,23))
plt.hlines(threshold,-1.0,23.0, "k", linewidth=3, linestyles="--", label="original failed transaction ratio")
plt.legend()
plt.savefig("/mnt/idms/fberes/data/bitcoin_ln_research/results/entity_failure_ratios.pdf", format="pdf", bbox_inches='tight')

# 6. Privacy preserving routing

In [ ]:
x = [2,3,4,5,6]
means = [5.0162, 5.1656, 5.2844, 5.4636, 5.6931]
medians = [1.08, 1.12, 1.18, 1.24, 1.5]

In [ ]:
plt.plot(x,means, 'o-', markersize=15, linewidth=3)
plt.xlabel("Minimum path length")
plt.ylabel("Mean cost (satoshi)")
plt.savefig("/mnt/idms/fberes/data/bitcoin_ln_research/results/privacy_mean_cost.pdf", format="pdf", bbox_inches='tight')

In [ ]:
plt.plot(x,medians, 'gs-', markersize=15, linewidth=3)
plt.xlabel("Minimum path length")
plt.ylabel("Median cost (satoshi)")
plt.ylim((1.0,1.6))
plt.savefig("/mnt/idms/fberes/data/bitcoin_ln_research/results/privacy_median_cost.pdf", format="pdf", bbox_inches='tight')

# 7. LNBIG.com tunning

In [ ]:
sim_dirs = {
    "100000sat_k7000_eps0.80",
    "20000sat_k7000_eps0.80",
    "40000sat_k7000_eps0.80",
    "60000sat_k11000_eps0.80",
    "60000sat_k3000_eps0.80",
    "60000sat_k5000_eps0.80",
    "60000sat_k7000_eps0.00",
    "60000sat_k7000_eps0.20",
    "60000sat_k7000_eps0.40",
    "60000sat_k7000_eps0.60",
    "60000sat_k7000_eps0.80",
    "60000sat_k7000_eps1.00",
    "60000sat_k9000_eps0.80",
    "80000sat_k7000_eps0.80"
}
root_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/lnbig_tuning_1days//"

In [ ]:
parts = []
for folder in sim_dirs:
    df = pd.read_csv("%s/%s_selected.csv" % (root_dir, folder))
    splitted = folder.split("_")
    amount = float(splitted[0].replace("sat",""))
    traffic = float(splitted[1].replace("k",""))
    eps = float(splitted[2].replace("eps",""))
    df["alpha"] = amount
    df["tau"] = traffic
    df["epsilon"] = eps
    parts.append(df.replace(node_name_replacements).rename({"mean_fee":"mean_income","mean_num_trans":"mean_traffic"},axis=1))
param_tuning_df = pd.concat(parts)

In [ ]:
param_tuning_df = param_tuning_df[param_tuning_df["name"]=="LNBIG.com"]

In [ ]:
param_tuning_df.head()

## Changing transaction amount

In [ ]:
df = param_tuning_df[(param_tuning_df["tau"]==7000) & (param_tuning_df["epsilon"]==0.8)]
df = df.sort_values("alpha")
x = list(df["alpha"])
y1, y2 = list(df["mean_income"]), list(df["mean_traffic"])

In [ ]:
fig = plt.figure(figsize=(18,5))
fig.suptitle(r'$\tau=7000, \epsilon=0.8$')

ax1 = fig.add_subplot(121)
ax1.plot(x,y1,'bs-',markersize=15,linewidth=3)
ax1.set_ylabel("income (satoshi)")
ax1.vlines(60000, min(y1), max(y1), "k", linewidth=3, linestyles=":")
for val in [5000, 10000]:
    ax1.hlines(val, x[0], x[-1], "r", linewidth=3, linestyles="--")

ax2 = fig.add_subplot(122)
ax2.plot(x,y2,'go-',markersize=15,linewidth=3)
ax2.set_ylabel("routed payments")
ax2.vlines(60000, 200, max(y2), "k", linewidth=3, linestyles=":")
for val in [200, 300]:
    ax2.hlines(val, x[0], x[-1], "r", linewidth=3, linestyles="--")

ax1.set_xlabel(r"$\alpha$ (satoshi)")
ax2.set_xlabel(r"$\alpha$ (satoshi)")

plt.savefig("/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/lnbig_tuning_1days/alpha_tuning.pdf", format="pdf", bbox_inches='tight')

## Changing transaction count

In [ ]:
df = param_tuning_df[(param_tuning_df["alpha"]==60000) & (param_tuning_df["epsilon"]==0.8)]
df = df.sort_values("tau")
x = list(df["tau"])
y1, y2 = list(df["mean_income"]), list(df["mean_traffic"])

In [ ]:
fig = plt.figure(figsize=(18,5))
fig.suptitle(r'$\alpha=60000, \epsilon=0.8$')

ax1 = fig.add_subplot(121)
ax1.plot(x,y1,'bs-',markersize=15,linewidth=3)
ax1.set_ylabel("income (satoshi)")
plt.xticks(x,[int(val) for val in x])
ax1.vlines(7000, min(y1), max(y1), "k", linewidth=3, linestyles=":")
for val in [5000, 10000]:
    ax1.hlines(val, x[0], x[-1], "r", linewidth=3, linestyles="--")

ax2 = fig.add_subplot(122)
ax2.plot(x,y2,'go-',markersize=15,linewidth=3)
ax2.set_ylabel("routed payments")
ax2.vlines(7000, min(y2), max(y2), "k", linewidth=3, linestyles=":")
plt.xticks(x,[int(val) for val in x])
for val in [200, 300]:
    ax2.hlines(val, x[0], x[-1], "r", linewidth=3, linestyles="--")

ax1.set_xlabel(r"$\tau$")
ax2.set_xlabel(r"$\tau$")

plt.savefig("/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/lnbig_tuning_1days/tau_tuning.pdf", format="pdf", bbox_inches='tight')

## Changing epsilon

In [ ]:
df = param_tuning_df[(param_tuning_df["alpha"]==60000) & (param_tuning_df["tau"]==7000)]
df = df.sort_values("epsilon")
x = list(df["epsilon"])
y1, y2 = list(df["mean_income"]), list(df["mean_traffic"])

In [ ]:
fig = plt.figure(figsize=(18,5))
fig.suptitle(r"$\alpha=60000, \tau=7000$")

ax1 = fig.add_subplot(121)
ax1.plot(x,y1,'bs-',markersize=15,linewidth=3)
ax1.set_ylabel("income (satoshi)")
ax1.vlines(0.8, 5000, 10000, "k", linewidth=3, linestyles=":")
for val in [5000, 10000]:
    ax1.hlines(val, x[0], x[-1], "r", linewidth=3, linestyles="--")

ax2 = fig.add_subplot(122)
ax2.plot(x,y2,'go-',markersize=15,linewidth=3)
ax2.set_ylabel("routed payments")
ax2.vlines(0.8, 200, 600, "k", linewidth=3, linestyles=":")
for val in [200, 300]:
    ax2.hlines(val, x[0], x[-1], "r", linewidth=3, linestyles="--")

ax1.set_xlabel(r"$\epsilon$")
ax2.set_xlabel(r"$\epsilon$")

plt.savefig("/mnt/idms/fberes/data/bitcoin_ln_research/results/router_traffic_and_income/lnbig_tuning_1days/epsilon_tuning.pdf", format="pdf", bbox_inches='tight')